In [5]:
import zipfile

zip_path = "/home/swc/Downloads/coords_Non.zip"

In [6]:
extract = "yolov9/data"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract)
print("Done")

Done


In [7]:
import pandas as pd

df= pd.read_csv("/home/swc/Downloads/yolov9/yolov9/data/coords_Non.csv")

In [15]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,Hold,0.407733,0.129086,-0.778690,0.993342,0.421445,0.090329,-0.737180,0.993019,0.434254,...,1.196196,0.036033,0.602610,1.653399,0.797693,0.050220,0.459393,1.667632,0.860123,0.037651
1,Hold,0.405080,0.119946,-0.982599,0.993911,0.419926,0.084866,-0.928936,0.993197,0.433784,...,1.411308,0.032501,0.603216,1.712780,0.894759,0.045268,0.460429,1.719580,1.059595,0.033972
2,Hold,0.397699,0.109015,-0.836984,0.994504,0.419621,0.072088,-0.792933,0.993758,0.433126,...,1.335354,0.029259,0.602546,1.728418,0.885714,0.040747,0.461573,1.751991,0.849992,0.030583
3,Hold,0.395727,0.105420,-0.799126,0.994938,0.418676,0.067525,-0.756440,0.993735,0.432624,...,1.248703,0.026352,0.604433,1.770216,0.844039,0.036681,0.465712,1.787531,0.881813,0.027538
4,Hold,0.394592,0.100825,-0.968101,0.995398,0.418038,0.063968,-0.916498,0.994151,0.432285,...,1.321547,0.023801,0.616181,1.812567,0.855019,0.033058,0.480407,1.833931,0.951099,0.024838


In [11]:
%pip install tensorflow

     |████████████████████████████████| 479.6 MB 38 kB/s s eta 0:00:01
     |████████████████████████████████| 5.6 MB 10.5 MB/s eta 0:00:01
     |████████████████████████████████| 24.5 MB 10.6 MB/s eta 0:00:01
     |████████████████████████████████| 440 kB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 9.8 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 5.9 MB/s  eta 0:00:01
     |████████████████████████████████| 6.0 MB 10.5 MB/s eta 0:00:01
     |████████████████████████████████| 106 kB 11.9 MB/s eta 0:00:01
     |████████████████████████████████| 227 kB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 210 kB 11.7 MB/s eta 0:00:01
ERROR: torch 2.4.1 has requirement typing-extensions>=4.8.0, but you'll have typing-extensions 4.5.0 

In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
def process_data(df,frames = 30,test = 0.2,random=42):
    print("Processing data")
    df= pd.read_csv("/home/swc/Downloads/yolov9/yolov9/data/coords_Non.csv",sep=",")

    class_columns = df.columns[0]
    labels = df[class_columns].unique()
    label_encoder = LabelEncoder()
    numeric_labels = label_encoder.fit_transform(labels)
    class_names = label_encoder.classes_
    print(f"Found {len(class_names)} classes: {class_names}")


    features_df = df.drop(columns=[class_columns])

    all_columns = features_df.columns.tolist()

    num_features = len(all_columns)

    print(f"Total features: {num_features}")

    missing_values = features_df.isnull().sum().sum()

    print(f"Missing: {missing_values}")

    if missing_values > 0:
        features_df.dropna(inplace=True)
        print("Filled missing values")


    print("Normalizing data")

    scaler = MinMaxScaler()

    normalized_features = scaler.fit_transform(features_df)

    print(f"organizing data {frames}")

    num_frames = len(normalized_features) - frames

    num_complete_sequences = num_frames // frames

    total_frames_used = num_complete_sequences * frames
    trimmed_features = normalized_features[:total_frames_used]
    trimmed_labels = numeric_labels[:total_frames_used]
    
    print(f"Using {total_frames_used} frames out of {num_frames} total frames")
    print(f"Creating {num_complete_sequences} complete sequences")

    X_sequences = np.zeros((num_complete_sequences, frames, num_features))
    y_sequences = np.zeros(num_complete_sequences)
    
    